# Ensemble Learning

## Initial Imports

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [ ]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay

## Read the CSV and Perform Basic Data Cleaning

In [ ]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

## Split the Data into Training and Testing

In [ ]:
# Create our features
X = df.drop(columns='loan_status')
X.head()

In [ ]:
X.shape


In [ ]:
# Need to turn the Ns etc. into binary
X = pd.get_dummies(X)
X.head()

In [ ]:
X.shape


In [ ]:
# Create our target
y = df['loan_status']
y = y.to_frame()
y.head()

In [ ]:
y.shape

 clearly imbalanced based on that ^^

In [ ]:
X.describe()

In [ ]:
# Check the balance of our target values
y['loan_status'].value_counts()

In [ ]:
# Split the X and y into X_train, X_test, y_train, y_test
# YOUR CODE HERE
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
X_train.head()


In [ ]:
X_test.head()


In [ ]:
y_train.head()


In [ ]:
y_test.head()

In [ ]:
y_test['loan_status'].value_counts()

In [ ]:
# As you can see above, data is really imbalanced. High Risk is far less likely to occur than low_risk

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [ ]:
# Create the StandardScaler instance
# YOUR CODE HERE
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler

In [ ]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
# YOUR CODE HERE
X_scaler = scaler.fit(X_train)
X_scaler


In [ ]:
# Scale the training and testing data
# YOUR CODE HERE
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
X_train_scaled[0]

In [ ]:
X_test_scaled[0]

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [ ]:
# Resample the training data with the BalancedRandomForestClassifier
# YOUR CODE HERE
from imblearn.ensemble import BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
brf.fit(X_train_scaled, y_train)

In [ ]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
y_predictions = brf.predict(X_test_scaled)
y_predictions

In [ ]:
rf_bac = balanced_accuracy_score(y_test, y_predictions)
print (f"the rf BAC is {rf_bac}")

In [ ]:
# Display the confusion matrix
# YOUR CODE HERE
cmx = confusion_matrix(y_test, y_predictions)
disp= ConfusionMatrixDisplay(confusion_matrix=cmx, display_labels=['high risk', 'low risk'])
disp.plot();

High Risk Recall: (71) / ( 71 + 30) = 0.7  <br>
Low Risk Recall: (14958) / (14958 + 2146) = .87 <br>
High Risk Precision: (71) / (71 + 2146) = 0.032 <br>
Low Risk Precision: (14958) / (14958 + 30) = .997

In [ ]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, y_predictions))

In [ ]:
# List the features sorted in descending order by feature importance
# YOUR CODE HERE
importances = brf.feature_importances_ 
importances

In [ ]:
importances_sorted = sorted(zip(brf.feature_importances_, X.columns), reverse=True)
importances_sorted[:10]

### Easy Ensemble Classifier

In [ ]:
# Train the Classifier
# YOUR CODE HERE

from imblearn.ensemble import EasyEnsembleClassifier
easyEnsemble = EasyEnsembleClassifier(n_estimators=100, random_state=1)
easyEnsemble.fit(X_train_scaled, y_train)
easyEnsemble

In [ ]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
ee_y_pred = easyEnsemble.predict(X_test_scaled)
ee_y_pred

In [ ]:
ee_bac = balanced_accuracy_score(y_test, ee_y_pred)
print (f"the EE BAC is {ee_bac}")

Tells us that we're 93% accurate overall at predicting true/accurate target values. <br>
^^ note, the balanced accuracy score could be meaningless if we're working with really imbalanced datasets, which we are.

In [ ]:
# This is the original confusion matrix
cmx = confusion_matrix(y_test, y_predictions)
disp= ConfusionMatrixDisplay(confusion_matrix=cmx, display_labels=['high risk', 'low risk'])
disp.plot();

In [ ]:
# Display the confusion matrix for the easy ensemble
# YOUR CODE HERE
cmx_ee = confusion_matrix(y_test, ee_y_pred)
disp= ConfusionMatrixDisplay(confusion_matrix=cmx_ee, display_labels=['high risk', 'low risk'])
disp.plot();

You can clearly see that the Easy Ensemble matrix has: <br>
> more true negatives, <br>
> more true negatives <br>
> less false positives, <br>
> less false negatives <br>


In [ ]:
# This is the classification report from the original RANDOM FOREST
print(classification_report_imbalanced(y_test, y_predictions))

In [ ]:
# Print the imbalanced classification report FOR THE EASY ENSEMBLE
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, ee_y_pred))

In [ ]:
#RANDOM FOREST
print (f"the rf BAC is {rf_bac}")

In [ ]:
# EASY ENSEMBLE
print (f"the EE BAC is {ee_bac}")

### Final Questions

1. Which model had the best balanced accuracy score?

    The Easy Ensemble

2. Which model had the best recall score?

    EASY ENSEMBLE, across the board. for high, risk and low risk, as well as the average / Total.

3. Which model had the best geometric mean score?

    EASY ENSEMBLE. 0.93 > 0.78. 

4. What are the top three features?

> 1. total_rec_prncp: 0.07876809003486353
> 2. total_pymnt: 0.05883806887524815
> 3. total_pymnt_inv: 0.05625613759225244